# Demo of Face Recognition Inference using Deep Learning
Copied from [ONNX ArcFace Example](https://github.com/onnx/models/blob/master/vision/body_analysis/arcface/dependencies/arcface_inference.ipynb) with a slight adjustment

**Overview**<br>
This notebook can be used for inference on ArcFace ONNX models. The demo shows how to use the trained models to do inference in MXNet.

**Models supported**<br>
LResNet100E-IR (ResNet100 backend with ArcFace loss)

**How To**<br>
1. Fill the image URLs you want to compare in `Download input images` section <br>
    <img src="image1.png" alt="fill-image-url" width="300" style="margin-left:0px" />
<br>
2. Click `Kernel` -> `Restart & Run All` to execute all commands in this notebook <br>
    <img src="image2.png" alt="run-all" width="400" style="margin-left:0px" />

<br>
<div style="width:550px; color:white; background-color:red; padding:5px; font-size:20px;"><b>WARNING!<br>The execution may take a long time (20min or more) because we have to download the model first<br>Leave this, do your other job, and check again later</b></div>

## Import dependencies

In [ ]:
import cv2
import os
import mxnet as mx
import numpy as np

from datetime import datetime
from mtcnn_detector import MtcnnDetector
from sklearn.preprocessing import normalize
from nanti_kita_pelajari_tentang_ini import \
    download_detection_model, get_recognition_model, get_input, get_feature, display_image

## Prepare the model

In [ ]:
# Determine and set context
if len(mx.test_utils.list_gpus())==0:
    ctx = mx.cpu()
else:
    ctx = mx.gpu(0)

In [ ]:
# Download ONNX model
model_dir = 'model-mtcnn'
model_path = os.path.join(os.path.dirname('__file__'), model_dir)
mx.test_utils.download(
    dirname=model_dir, url='https://s3.amazonaws.com/onnx-model-zoo/arcface/resnet100.onnx')

In [ ]:
# Load ONNX model
model_name = os.path.join(model_path, 'resnet100.onnx')
model = get_recognition_model(ctx , model_name)

In [ ]:
# Configure face detector
download_detection_model('https://s3.amazonaws.com/onnx-model-zoo/arcface/mtcnn-model', model_dir)
det_threshold = [0.6,0.7,0.8]
detector = MtcnnDetector(
    model_folder=model_path, ctx=ctx, num_worker=1, accurate_landmark = True, threshold=det_threshold)

## Download input images

In [ ]:
# Please fill this first
url1 = 
url2 = 

# Download images
url = [url1, url2]
for u in url:
  mx.test_utils.download(u)

# Load images
img1 = cv2.imread(url1.split("/")[-1])
img2 = cv2.imread(url2.split("/")[-1])

In [ ]:
# Display first image
display_image(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))

In [ ]:
# Display second image
display_image(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

## Preprocess images

In order to input only face pixels into the network, all input images are passed through a pretrained face detection and alignment model.<br>
The output of this model are landmark points and a bounding box corresponding to the face in the image.<br>
Using this output, the image is processed using affine transforms to generate the aligned face images which are input to the network.<br>
The functions performing this can be checked further in `get_input`.

In [ ]:
# Detect face & align in first image
%time pre1 = get_input(detector, img1)
display_image(np.transpose(pre1, (1,2,0)))

In [ ]:
# Detect face & align in second image
%time pre2 = get_input(detector, img2)
display_image(np.transpose(pre2, (1,2,0)))

## Generate predictions
Two face images are passed through the network sequentially to generate embedding vectors for each.

In [ ]:
# Get embedding of first image & normalize
%time out1 = get_feature(model, pre1)
out1 = normalize(out1).flatten()

# Get embedding of second image & normalize
%time out2 = get_feature(model, pre2)
out2 = normalize(out2).flatten()

In [ ]:
# Check normal
assert np.sum(np.square(out1)) - 1. < 1e-6
assert np.sum(np.square(out2)) - 1. < 1e-6

In [ ]:
# Compute [?] distance between embeddings
dist = np.sum(np.square(out1-out2))
print(dist)

In [ ]:
# Compute [?] distance between embeddings
dist = np.sum(np.abs(out1-out2))
print(dist)

In [ ]:
# Compute [?] distance between embedddings
dist = np.dot(out1, out2.T)
print(dist)

# Check also https://en.wikipedia.org/wiki/Dot_product

In [ ]:
# Use this if you don't want to convert the distance
threshold = np.cos(np.pi/4)

if dist > threshold:
    print("Loh kok mirip? Kembar kah?")
else:
    print("Ga mirip ah, ngaku-ngaku ya?")

In [ ]:
# Above checking is simlar to,
threshold = np.pi/4
distance = np.arccos(dist)

if distance < threshold:
    print("Loh kok mirip? Kembar kah?")
else:
    print("Ga mirip ah, ngaku-ngaku ya?")